In [58]:
import re, mendeleev
from collections import Counter
import numpy as np
import math
import ipywidgets as widgets
from IPython.display import display, clear_output

In [49]:
ATOM_REGEX = '([A-Z][a-z]*)(\d*\.*\d*)'
OPENERS = '({['
CLOSERS = ')}]'

def parse_molecule(formula):
    """Parse the formula and return a dict with occurences of each atom.
    @param formula: The formula to parse
    @type formula: str
    @return: The result of the forumla
    @rtype : dict
    @raise ValueError: The formula is invalid
    """
    """Parse the formula and return a dict with occurences of each atom"""
    if not is_balanced(formula):
        raise ValueError("Watch your brackets ![{]$[&?)]}!]")
    return parse(formula)[0]


def is_balanced(formula):
    """Check if all sort of brackets come in pairs.
    @param formula: The forumla to parse
    @type formula: str
    @return: Result of sort of brackets come in pairs
    @rtype : bool
    """
    counter = Counter(formula)
    result = counter[
        '['] == counter[']'] and counter['{'] == counter[
            '}'] and counter['('] == counter[')']
    return(result)

def dictify(tuples):
    res = {}
    for atom, n in tuples:
        if atom in res:
            res[atom] += float(n or 1)
        else:
            res[atom] = float(n or 1)
    return(res)

def fuse(mol1, mol2, w=1):
    return( {
        atom: (
            mol1.get(atom, 0) + mol2.get(atom, 0)) * w
        for atom in set(mol1) | set(mol2)})

def parse(formula):
    element = []
    mol = {}
    index = 0
    while index < len(formula):
        token = formula[index]
        if token in CLOSERS:
            match = re.match("\d*\.*\d*", formula[index+1:])
            if match:
                weight = float(match.group(0))
                index += len(match.group(0))
            else:
                weight = 1
            submol = dictify(re.findall(ATOM_REGEX, ''.join(element)))
            return fuse(mol, submol, weight), index
        elif token in OPENERS:
            submol, count = parse(formula[index+1:])
            mol = fuse(mol, submol)
            index += count + 1
        else:
            element.append(token)
        index += 1
    return fuse(
        mol, dictify(re.findall(ATOM_REGEX, ''.join(element)))), index


def calculate_rmm(formula):
    rmm = 0
    for i in formula:
        rmm += mendeleev.element(i).mass * formula[i]
    return(rmm)

def calculate_moles(target_mass, rmm):
    return(target_mass/rmm)

def Merge(dict1, dict2): 
    res = {**dict1, **dict2} 
    return res 



In [50]:
def calculate_reagents(reagent_list, moles):
    reagent_list = {i[0]:i[1] for i in reagent_list}
    reagents_out = []
    for i in reagent_list.keys():
        formula = parse_molecule(i)
        rmm = calculate_rmm(formula)
        moles = float(reagent_list[i]) * moles
        reagents_out.append(i)
        reagents_out.append("RMM: {:.4f}".format(rmm))
        reagents_out.append("Moles: {:e}".format(moles))
        reagents_out.append("Mass required: {:.4f} g".format(moles * rmm))
        reagents_out.append("-"*10)
    return "\n".join(reagents_out)

def format_equation(reagent_list, target):
    reagents_list = ["({}){}".format(i[0],i[1]) for i in reagent_list]
    reagents_eq = " + ".join(reagents_list)
    equation = " --> ".join([str(i) for i in [reagents_eq, target]])
    return equation

def check_calculation(reagent_list, formula):
    reagents_check = ["({}){}".format(i[0],i[1]) for i in reagent_list]
    reagents_check = parse_molecule("".join(reagents_check))
    if reagents_check == formula:
        out = "Equation is balanced."
    else:
        out = "EQUATION IS NOT BALANCED"
    return out


In [51]:
#!jupyter nbextension enable --py widgetsnbextension --sys-prefix
#!jupyter serverextension enable voila --sys-prefix

In [52]:
text_0 = widgets.HTML(value = "<h1>Reagents Calculator</h1>")
text_1 = widgets.HTML(value = "Target composition:")
text_2 = widgets.HTML(value = "Target mass in grams:")
text_3 = widgets.HTML(value = "Reagents and molar equivalents<br>(one per line separated by a space)")

In [53]:
target_comp = widgets.Text(placeholder = "e.g. CaTiO3")
target_mass = widgets.Text(placeholder = "e.g. 1")
reagents = widgets.Textarea(placeholder= "e.g.\nCaCO3 1\nTiO2 1")

In [54]:
output = widgets.Output()

button_calculate = widgets.Button(
                description = "Calculate!",
                style = {"description_width": "initial"}
            )


def on_button_clicked(event):
    with output:
        output.clear_output()
        target = target_comp.value
        formula = parse_molecule(target)
        reagent_list = [i.split(" ") for i in reagents.value.split("\n")]
        print(format_equation(reagent_list, target))
        #print("Parsed formula:")
        #print(formula)
        print(check_calculation(reagent_list, formula))
        rmm = calculate_rmm(formula)
        print("Calculated RMM:")
        print("{:.4f}".format(rmm))
        print("Calculated Moles:")
        mass = float(target_mass.value)
        moles = calculate_moles(mass, rmm)
        print("{:.4f} / {:.4f} = {:e}".format(mass,
                                              rmm,
                                              moles))
        print("-"*20)
        #print(reagent_list)
        print(calculate_reagents(reagent_list, moles))
    
button_calculate.on_click(on_button_clicked)

vbox_result = widgets.VBox([output])

In [55]:
vbox_head = widgets.VBox([text_0,
                            text_1,
                            target_comp,
                            text_2,
                            target_mass,
                            text_3,
                            reagents,
                            button_calculate])

In [56]:
page = widgets.HBox([vbox_head, vbox_result])
display(page)